<a href="https://colab.research.google.com/github/michealman114/NLP_Hate_Speech_Detection/blob/main/Hate_Speech_Detection_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install textblob
!pip install pyspellchecker
!pip install nltk
!pip install tqdm

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ac4afc512847715e94ef1df29a997b7afc70e479886ac36a1fc5a25aa445d7b1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 2.7 MB 4.6 MB/s 


In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data

In [ ]:
import numpy as np
import wget
import json

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

from textblob import TextBlob 
import re

from spellchecker import SpellChecker

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from torch import cuda

seed = 4814

if cuda.is_available():
  device = 'cuda'
  torch.cuda.manual_seed_all(seed)
else:
  print('running on cpu')
  device = 'cpu'

torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
data = wget.download('https://raw.githubusercontent.com/sjtuprog/fox-news-comments/master/fox-news-comments.json')

modern_data = wget.download('https://raw.githubusercontent.com/michealman114/NLP_Hate_Speech_Detection/main/modern_comments.json')

print(data)
print(modern_data)

fox-news-comments.json
modern_comments.json


In [ ]:
!ls

fox-news-comments.json	modern_comments.json  sample_data


In [ ]:
train_lines = open("fox-news-comments.json", "r").readlines()
test_lines = open("modern_comments.json", "r").readlines()

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.add('')

from nltk.tokenize import word_tokenize

In [ ]:
def clean(data):
  max_len = 0
  max_title_len = 0  
  text_list = []
  title_list = []
  label = []
  for i in data:
    comment = json.loads(i)
    t = comment['text']
    t = ' '.join([x for x in t.split() if x[0] != '@'])
    t = ' '.join(re.findall("[a-zA-Z,.]+",t))
    t = t.replace(',', ' ')
    t = t.replace('.', ' ')
    text = word_tokenize(t)
    text = [x for x in text if x.lower() not in stop_words]
    max_len = max(max_len, len(text))
    text_list.append(text)
    title = comment['title']
    title = title.replace(',', '')
    title = title.replace('.', '')
    title = re.findall("[a-zA-Z,.]+",title)
    title_list.append(title)
    max_title_len = max(max_title_len, len(title))
    label.append(comment['label'])
  
  labels = np.array(label)
  return text_list, labels, title_list, max_len, max_title_len

train_text, train_labels, train_title, train_max_len, train_max_title_len = clean(train_lines)
test_text, test_labels, test_title, test_max_len, test_max_title_len = clean(test_lines)


In [ ]:
spell = SpellChecker()
spell.word_frequency.add('obama')
spell.word_frequency.add('blm')
spell.word_frequency.add('killing')

In [ ]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
import gensim
embed = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
def get_embed(word):
  x = np.zeros((300,)) # default value should be 0
  corrected = spell.correction(word) # closest correction
  if word in embed: # base word
    x = embed[word]
  elif word.upper() in embed: # capitalized (edge case for acronyms like BLM - blm doesn't exist but BLM does)
    x = embed[word.upper()]
  elif word.lower() in embed: # opposite of capitalization
    x = embed[word.lower()]
  elif corrected in embed: # last case, check if closest correction exists (some corrections are not great)
    x = embed[corrected]
  
  return x

In [ ]:
print('comments',train_max_len)
print('title',train_max_title_len)
print(len(train_text))
print(len(train_title))
print(train_labels.shape)

comments 244
title 13
1528
1528
(1528,)


In [ ]:
print('test_comments', test_max_len)
print('test_title', test_max_title_len)
print(len(test_text))
print(len(test_title))
print(test_labels.shape)

test_comments 126
test_title 19
102
102
(102,)


In [ ]:
def to_array(comments, titles, max_comment_len, max_title_len):
  data_array = np.zeros((max_comment_len, len(comments), 300))
  title_array = np.zeros((max_title_len, len(titles), 300))
  for ix1, sent in enumerate(comments):
    for ix2, word in enumerate(sent):
      data_array[ix2,ix1] = get_embed(word)
  for ix1, title in enumerate(titles):
    for ix2, word in enumerate(title):
      title_array[ix2,ix1] = get_embed(word)
  
  return data_array, title_array

train_data_array, train_title_array = to_array(train_text, train_title, train_max_len, train_max_title_len)
test_data_array, test_title_array = to_array(test_text, test_title, test_max_len, test_max_title_len)

In [ ]:
print(train_data_array.shape, train_title_array.shape)
print(test_data_array.shape, test_title_array.shape)

(244, 1528, 300) (13, 1528, 300)
(126, 102, 300) (19, 102, 300)


In [ ]:
"""
data_array.shape = (244, 1528, 300)
data_labels.shape = (1528,)
data is an (L,N,D) array
L = max_length of sequence
N = batch_size
D = embed_dim
"""

#Source: https://likegeeks.com/numpy-shuffle/#Shuffle_multiple_NumPy_arrays_together
def custom_shuffle(data,titles,labels):
    _, num_samples, _ = data.shape
    shuffled_indices = np.random.permutation(num_samples) #return a permutation of the indices
    new_data = data[:,shuffled_indices,:]
    new_labels = labels[shuffled_indices]
    new_titles = titles[:,shuffled_indices,:]

    return (new_data, new_titles, new_labels)
    
train_data_array,train_title_array,train_labels = custom_shuffle(train_data_array,train_title_array,train_labels)

test_data_array, test_title_array, test_labels = custom_shuffle(test_data_array, test_title_array, test_labels)

In [ ]:
print(train_data_array[:,0,:].shape)

(244, 300)


In [ ]:
print(train_data_array.dtype)
print(type(train_data_array))
print(train_title_array.dtype)
print(type(train_title_array))

data_array = np.float32(train_data_array)
title_array = np.float32(train_title_array)

test_data_array = np.float32(test_data_array)
test_title_array = np.float32(test_title_array)

float64
<class 'numpy.ndarray'>
float64
<class 'numpy.ndarray'>


In [ ]:
class BaseModel(nn.Module): # single direction lstm, no attention
  def __init__(self, hidden_size = 100, embed_dim = 300):
    super(BaseModel, self).__init__()
    
    self.hidden_size = hidden_size
    
    self.linear1 = nn.Linear(hidden_size, 150) # map context vector to value
    self.linear2 = nn.Linear(150, 1)

    self.relu = nn.ReLU()

    self.sigmoid = nn.Sigmoid()
    
    self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = 1, batch_first = False, dropout = 0.2, bidirectional = False)

  def forward(self, data):
    """
    data is an (L,N,D) array
    L = max_length of sequence
    N = batch_size
    D = embed_dim
    returns an (N,1) array of probabilities that each comment is hateful
    """
    hidden_states, (_, _) = self.lstm(data) # (L,N,H) array
    
    sentences = torch.sum(hidden_states, axis = 0)

    return self.sigmoid(torch.squeeze(self.linear2(self.relu(self.linear1(sentences)))))

In [ ]:
class BidiModel(nn.Module): # Bidi
  def __init__(self, hidden_size = 100, embed_dim = 300):
    super().__init__()
    
    self.hidden_size = hidden_size
    
    self.linear1 = nn.Linear(2*hidden_size, hidden_size) # map context vector to value
    self.linear2 = nn.Linear(hidden_size, 1)

    self.relu = nn.ReLU()

    self.sigmoid = nn.Sigmoid()
    
    self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = 1, batch_first = False, dropout = 0.2, bidirectional = True)

  def forward(self, data):
    """
    data is an (L,N,D) array
    L = max_length of sequence
    N = batch_size
    D = embed_dim
    returns an (N,1) array of probabilities that each comment is hateful
    """
    hidden_states, (_, _) = self.lstm(data) # (L,N,2H) array

    sentences = torch.sum(hidden_states, axis = 0)

    return self.sigmoid(torch.squeeze(self.linear2(self.relu(self.linear1(sentences)))))

In [ ]:
class FullModel(nn.Module): # bidi with attention
  def __init__(self, hidden_size = 100, embed_dim = 300):
    super().__init__()
    
    self.hidden_size = hidden_size
    #self.embedding = embed
    
    self.linear1 = nn.Linear(2*hidden_size, hidden_size) # map context vector to value
    self.linear2 = nn.Linear(hidden_size, 1)

    self.attention1 = nn.Linear(2*hidden_size, 50) # map hidden state vector to value
    self.attention2 = nn.Linear(50, 1)

    self.relu = nn.ReLU()

    self.sm = nn.Softmax(dim = 0)
    self.sigmoid = nn.Sigmoid()
    
    self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = 1, batch_first = False, dropout = 0.2, bidirectional = True)

  def forward(self, data):
    """
    data is an (L,N,D) array
    L = max_length of sequence
    N = batch_size
    D = embed_dim
    returns an (N,1) array of probabilities that each comment is hateful
    """
    hidden_states, (_, _) = self.lstm(data) # (L,N,2H) array
    weights = self.attention2(self.relu(self.attention1(hidden_states))) # (L,N,1) array
    
    alpha = self.sm(weights.reshape(weights.shape[:-1])) # weights

    hidden_states = torch.moveaxis(hidden_states, -1, 0)


    sentences = torch.sum(hidden_states * alpha, axis = 1)

    sentences = torch.moveaxis(sentences, 0, -1)

    return self.sigmoid(torch.squeeze(self.linear2(self.relu(self.linear1(sentences)))))

In [ ]:
class ModelWithTitle(nn.Module): # bidi with attention
  def __init__(self, hidden_size = 100, embed_dim = 300):
    super().__init__()
    
    self.hidden_size = hidden_size
    
    self.linear1 = nn.Linear(4*hidden_size, hidden_size) # map context vector to value (concatenated from parallel networks)
    self.linear2 = nn.Linear(hidden_size, 1)

    self.relu = nn.ReLU()

    self.sm = nn.Softmax(dim = 0)
    self.sigmoid = nn.Sigmoid()

    #comments
    self.attention1_comment = nn.Linear(2*hidden_size, 50) # map hidden state vector to value
    self.attention2_comment = nn.Linear(50, 1)
    
    self.lstm_comment = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = 1, batch_first = False, dropout = 0.2, bidirectional = True)

    #titles
    self.attention1_title = nn.Linear(2*hidden_size, 50)
    self.attention2_title = nn.Linear(50, 1)

    self.lstm_title = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = 1, batch_first = False, dropout = 0.2, bidirectional = True)

  def forward(self, comment_data, title_data):
    """
    comments is an (L1,N,D) array
    titles is an (L2,N,D) array
    L1 = max_length of sequence
    L2 = max_length of title
    N = batch_size
    D = embed_dim
    returns an (N,1) array of probabilities that each comment is hateful
    """
    hidden_states, (_, _) = self.lstm_comment(comment_data) # (L,N,2H) array

    weights = self.attention2_comment(self.relu(self.attention1_comment(hidden_states))) # (L,N,1) array
    
    alpha = self.sm(weights.reshape(weights.shape[:-1])) # weights

    hidden_states = torch.moveaxis(hidden_states, -1, 0)

    sentences = torch.sum(hidden_states * alpha, axis = 1)

    sentences = torch.moveaxis(sentences, 0, -1) # sentences is N x 2*hidden_size

    hidden_states, (_,_) = self.lstm_title(title_data)
    weights = self.attention2_title(self.relu(self.attention1_title(hidden_states)))
    alpha = self.sm(weights.reshape(weights.shape[:-1]))
    hidden_states = torch.moveaxis(hidden_states, -1, 0)
    titles = torch.sum(hidden_states * alpha, axis = 1)
    titles = torch.moveaxis(titles, 0, -1) # titles is N x 2*hidden_size

    result = torch.cat((sentences, titles), dim = 1)

    return self.sigmoid(torch.squeeze(self.linear2(self.relu(self.linear1(result)))))

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, data, labels, titles = None):
        # initialization now works with titles, passes in optional title information
        # works the same as before, but now gets title data if you give it to it
        'Initialization'
        self.data = data
        self.titles = titles
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = self.data[:,ID,:]
        y = self.labels[ID]

        if self.titles is not None:
          t = self.titles[:,ID,:]
          return X, t, y
        return X, y

In [ ]:
from tqdm import tqdm

def train(data, labels, n_epochs, batch_size, modeltype, model = None):
    device = torch.device('cuda')  # run on colab gpu

    if model is None:
        model = modeltype().to(device)
        
    opt = optim.Adam(model.parameters(), lr=0.001)

    training_data = Dataset(range(len(labels)), data, labels)

    loader = torch_data.DataLoader(training_data, batch_size=batch_size, shuffle=True)

    loss_fn = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        epoch_loss = 0
        for context, label in loader:
            context = context.to(device)
            context = context.moveaxis(0, 1)
            label = label.to(device).type(torch.float32)

            preds = model.forward(context)
            #print(preds)
            #print(label)

            opt.zero_grad()
            loss = loss_fn(preds, label)
            loss.backward()
            opt.step()

            epoch_loss += loss.item()
        print('Loss:', epoch_loss)
        losses.append(epoch_loss)

    print(losses)
    return model

In [ ]:
def train_with_titles(data, titles, labels, n_epochs, batch_size, model = None):
    device = torch.device('cuda')  # run on colab gpu

    if model is None:
        model = ModelWithTitle().to(device)
        
    opt = optim.Adam(model.parameters(), lr=0.001)

    training_data = Dataset(range(len(labels)), data, labels, titles = titles)

    loader = torch_data.DataLoader(training_data, batch_size=batch_size, shuffle=True)

    loss_fn = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        epoch_loss = 0
        for context, t, label in loader:
            context = context.to(device)
            context = context.moveaxis(0, 1)
            t = t.to(device)
            t = t.moveaxis(0,1)
            label = label.to(device).type(torch.float32)

            preds = model.forward(context, t)
            #print(preds)
            #print(label)

            opt.zero_grad()
            loss = loss_fn(preds, label)
            loss.backward()
            opt.step()

            epoch_loss += loss.item()
        print('Loss:', epoch_loss)
        losses.append(epoch_loss)

    print(losses)
    return model

In [ ]:
print(data_array.shape)
model = train_with_titles(train_data_array, train_title_array, train_labels, 30, 128)

In [ ]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
"""
data_array.shape = (244, 1528, 300)
data_labels.shape = (1528,)
data is an (L,N,D) array
L = max_length of sequence
N = batch_size
D = embed_dim
"""

def kfold_crossvalidation(data, labels, modeltype, k, n_epochs = 30, model=None):
    _, num_samples, _ = data.shape
    fraction = 1/k
    seg = int(num_samples * fraction)
    segment_indices = []
    for i in range(k):
        vall = i * seg
        valr = i * seg + seg
        segment_indices.append(list(range(vall,valr)))
    
    all_preds = []
    all_labels = []
    #actually run the ith split
    for i in range(k):
        train_indices = []
        test_indices = segment_indices[i]
        for j in range(k):
            if j != i:
                train_indices.extend(segment_indices[j])

    
        train_data = data[:,train_indices,:]
        train_labels = labels[train_indices]

        test_data = data[:,test_indices,:]
        test_labels = labels[test_indices]

        batch_size = 128
        model_i = train(train_data, train_labels, n_epochs, batch_size, modeltype)

        iter_loss, (y_pred, y_true) = test_model(model_i,test_data,test_labels)
        y_pred = torch.round(y_pred).cpu().detach().numpy()
        all_preds.append(y_pred)
        all_labels.append(y_true)
        #print(type(y_true), y_true.shape)
        #print(type(y_pred), y_pred.shape)
        print('Accuracy:',accuracy_score(y_true,y_pred))
        print('Precision, Recall, F1:',precision_recall_fscore_support(y_true, y_pred, average='binary'))
    
    print('\n===Aggregate Stats===')
    p = np.concatenate(all_preds, axis = None)
    l = np.concatenate(all_labels, axis = None)
    print('Accuracy:', accuracy_score(l, p))
    print('Precision, Recall, F1:', precision_recall_fscore_support(l, p, average = 'binary'))

def test_model(model, test_data, test_labels):
    test_dataset = Dataset(range(len(test_labels)), test_data, test_labels)

    test_loader = torch_data.DataLoader(test_dataset, batch_size=len(test_labels))
    loss_fn = nn.BCELoss()

    predictions = None
    
    for context, label in test_loader:
        context = context.to(device)
        context = context.moveaxis(0, 1)
        label = label.to(device).type(torch.float32)

        #preds is a tensor of roughly torch.Size([305])
        preds = model.forward(context)
        predictions = preds

        loss = loss_fn(preds, label)

        print(loss.item())
    

    return loss.item(), (preds, test_labels)


def test_split():
    #function to verify that the function was splitting the data correctly
    temp_data = np.random.rand(2,5,3)
    temp_labels = np.random.randint(2, size=5)
    print("data:\n", temp_data)
    print("labels:", temp_labels)

    kfold_crossvalidation(temp_data,temp_labels,10)

kfold_crossvalidation(train_data_array,train_labels, modeltype = FullModel, k = 10,n_epochs = 30, model = None)

In [ ]:
def kfold_crossvalidation_titles(data, titles, labels, k, n_epochs = 30, model=None):
    _, num_samples, _ = data.shape
    fraction = 1/k
    seg = int(num_samples * fraction)
    segment_indices = []
    for i in range(k):
        vall = i * seg
        valr = i * seg + seg
        segment_indices.append(list(range(vall,valr)))
    
    all_preds = []
    all_labels = []
    #actually run the ith split
    for i in range(k):
        train_indices = []
        test_indices = segment_indices[i]
        for j in range(k):
            if j != i:
                train_indices.extend(segment_indices[j])

    
        train_data = data[:,train_indices,:]
        train_titles = titles[:,train_indices,:]
        train_labels = labels[train_indices]

        test_data = data[:,test_indices,:]
        test_titles = titles[:,test_indices,:]
        test_labels = labels[test_indices]

        batch_size = 128
        model_i = train_with_titles(train_data, train_titles, train_labels, n_epochs, batch_size)

        iter_loss, (y_pred, y_true) = test_model_titles(model_i,test_data,test_titles,test_labels)
        y_pred = torch.round(y_pred).cpu().detach().numpy()
        all_preds.append(y_pred)
        all_labels.append(y_true)
        #print(type(y_true), y_true.shape)
        #print(type(y_pred), y_pred.shape)
        print('Accuracy:',accuracy_score(y_true,y_pred))
        print('Precision, Recall, F1:',precision_recall_fscore_support(y_true, y_pred, average='binary'))
    
    print('\n===Aggregate Stats===')
    p = np.concatenate(all_preds, axis = None)
    l = np.concatenate(all_labels, axis = None)
    print('Accuracy:', accuracy_score(l, p))
    print('Precision, Recall, F1:', precision_recall_fscore_support(l, p, average = 'binary'))

def test_model_titles(model, test_data, test_titles, test_labels):
    test_dataset = Dataset(range(len(test_labels)), test_data, test_labels, titles = test_titles)

    test_loader = torch_data.DataLoader(test_dataset, batch_size=len(test_labels))
    loss_fn = nn.BCELoss()

    predictions = None
    
    for context, t, label in test_loader:
        context = context.to(device)
        context = context.moveaxis(0, 1)
        t = t.to(device)
        t = t.moveaxis(0,1)
        label = label.to(device).type(torch.float32)

        #preds is a tensor of roughly torch.Size([305])
        preds = model.forward(context,t)
        predictions = preds

        loss = loss_fn(preds, label)

        print(loss.item())
    

    return loss.item(), (preds, test_labels)

kfold_crossvalidation_titles(train_data_array,train_title_array,train_labels,10,n_epochs = 40, model = None)

In [ ]:
trained_model = train(train_data_array, train_labels, 20, 128, modeltype = FullModel)
_, (p, l) = test_model(trained_model, test_data_array, test_labels)
p = torch.round(p).cpu().detach().numpy()
print('Accuracy:',accuracy_score(l,p))
print('Precision, Recall, F1:',precision_recall_fscore_support(l, p, average = 'binary'))

In [ ]:
trained_model = train_with_titles(train_data_array, train_title_array, train_labels, 20, 128)
_, (p, l) = test_model_titles(trained_model, test_data_array, test_title_array, test_labels)
p = torch.round(p).cpu().detach().numpy()
print('Accuracy:',accuracy_score(l,p))
print('Precision, Recall, F1:',precision_recall_fscore_support(l, p, average = 'binary'))